## Grid-Search



In [2]:
# Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
# GridSearch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [3]:

# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

print(X.shape)
print(y.shape)

(1797, 64)
(1797,)


In [4]:

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

In [5]:

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 
                     'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    
                    {'kernel': ['linear'], 
                     'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on train set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on train set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed classification report on test-set:")
    print()
    #print("The model is trained on the full development set.")
    #print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.


# Tuning hyper-parameters for precision



Best parameters set found on train set:

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on train set:

0.986 (+/-0.016) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.959 (+/-0.028) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.988 (+/-0.017) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.982 (+/-0.026) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.988 (+/-0.017) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.983 (+/-0.026) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.988 (+/-0.017) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.983 (+/-0.026) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.974 (+/-0.012) for {'C': 1, 'kernel': 'linear'}
0.974 (+/-0.012) for {'C': 10, 'kernel': 'linear'}
0.974 (+/-0.012) for {'C': 100, 'kernel': 'linear'}
0.974 (+/-0.012) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report on test-set:


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        89
      

## Random-Search

In [6]:
# RandomSearch CV
# Source: https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html

import numpy as np

from time import time
import scipy.stats as stats

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier

In [7]:

# get some data
X, y = load_digits(return_X_y=True, n_class=3)

# build a classifier
clf = SGDClassifier(loss='hinge', penalty='elasticnet',
                    fit_intercept=True)

In [8]:

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [9]:

# specify parameters and distributions to sample from
param_dist = {'average': [True, False],
              'l1_ratio': stats.uniform(0, 1),
              'alpha': stats.loguniform(1e-4, 1e0)}

# run randomized search
n_iter_search = 15
random_search = RandomizedSearchCV(clf, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 1.29 seconds for 15 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.989 (std: 0.014)
Parameters: {'alpha': 0.0006913806210326083, 'average': False, 'l1_ratio': 0.11043256340099361}

Model with rank: 2
Mean validation score: 0.989 (std: 0.007)
Parameters: {'alpha': 0.00012948402484271382, 'average': False, 'l1_ratio': 0.8849478512947113}

Model with rank: 3
Mean validation score: 0.987 (std: 0.014)
Parameters: {'alpha': 0.0014703603391795646, 'average': True, 'l1_ratio': 0.8497527615968151}



In [10]:

# use a full grid over all parameters
param_grid = {'average': [True, False],
              'l1_ratio': np.linspace(0, 1, num=10),
              'alpha': np.power(10, np.arange(-4, 1, dtype=float))}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)


GridSearchCV took 6.68 seconds for 100 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.994 (std: 0.005)
Parameters: {'alpha': 0.001, 'average': False, 'l1_ratio': 0.4444444444444444}

Model with rank: 2
Mean validation score: 0.993 (std: 0.009)
Parameters: {'alpha': 1.0, 'average': False, 'l1_ratio': 0.0}

Model with rank: 3
Mean validation score: 0.993 (std: 0.011)
Parameters: {'alpha': 0.001, 'average': False, 'l1_ratio': 0.1111111111111111}



## Bayesian Optimization

In [11]:
#HyperOPT: Bayesian Optimization

#Refer: https://hyperopt.github.io/hyperopt-sklearn/

In [12]:
#!pip install hpsklearn
#!pip install hyperopt

In [33]:
from hpsklearn import HyperoptEstimator, any_classifier
from sklearn.datasets import fetch_openml
from hyperopt import tpe
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

In [34]:

# Download the data and split into training and test sets
digits = fetch_openml('mnist_784', parser='auto', as_frame=False)

X = digits.data
y = digits.target

In [35]:
random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

In [36]:

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [37]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [39]:
y

array(['0', '4', '1', ..., '7', '1', '1'], dtype=object)

In [43]:

estim = HyperoptEstimator( classifier=any_classifier('clf'),  
                          algo=tpe.suggest, 
                          trial_timeout=300)

#estim.fit( X_train, y_train )

#print( estim.score( X_test, y_test ) )


In [ ]:
print( estim.best_model() )